# <center> MRI Segmentation

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# импортируем требуемые бибилиотеки
import torch
from source.network import NNSegmentation
from source.helpme import show_image, load_mri, create_loader, calculate_pad, show_aug_grid_segmentation
import numpy as np
import torchvision
import os
from torchvision import transforms
from sklearn.model_selection import train_test_split
import glob


# импортируем функции аугментации
from albumentations import (
    Compose, 
    RandomBrightnessContrast, 
    ShiftScaleRotate, 
    RandomSnow,
    Cutout,
    Flip,
    Transpose
)

[Ссылка](https://github.com/albu/albumentations#pixel-level-transforms) на полный список аугментаций

In [ ]:
# загрузим данные
X, y = load_mri(os.path.join('train1', 'train'), 
                size=(128, 128))

In [ ]:
# разобьем на трэин и тест
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.333, shuffle=True, random_state=17)

X_test = np.expand_dims(X_test, 1)

In [ ]:
# определим аугментацию
aug = Compose([
    
    Flip(p=0.5),
    Cutout(num_holes=5, max_h_size=5, max_w_size=5, fill_value=0, always_apply=False, p=0.5),
    ShiftScaleRotate(shift_limit=0.15, scale_limit=0.1, rotate_limit=35 , border_mode=0, p=0.5)

])

In [ ]:
# создадим train loader
train_loader = create_loader(X_train, y_train, trs = aug, shuffle=False, apply_to_targets=True)

In [ ]:
# посмотрим как выглядят картинки с примененной аугментацией
show_aug_grid_segmentation(train_loader, idx=0, size=5)

## U-net

In [ ]:
# определим архитектуру сети

unet = torch.nn.Sequential(torch.nn.Conv2d(in_channels=1, 
                                               out_channels=16, 
                                               kernel_size=3, 
                                               stride=2, 
                                               padding=1),
                               torch.nn.LeakyReLU(),
                               torch.nn.BatchNorm2d(16),
                               # 64x64

                               torch.nn.Conv2d(16, 32, 3, stride=2, padding=1),
                               torch.nn.LeakyReLU(),
                               torch.nn.BatchNorm2d(32),
                               # 32x32
                               
                               
                               torch.nn.MaxPool2d(kernel_size=3, 
                                                  stride=2, 
                                                  padding=1),
                               # 16x16

                               torch.nn.Conv2d(32, 64, 3, stride=2, padding=1),
                               torch.nn.LeakyReLU(),
                               torch.nn.BatchNorm2d(64),
                               # 8x8

                               torch.nn.Conv2d(64, 128, 3, stride=2, padding=1),
                               torch.nn.LeakyReLU(),
                               torch.nn.BatchNorm2d(128),
                               # 4x4

                               # далее обратно увеличиваем spatial size
                               torch.nn.ConvTranspose2d(128, 128, 4, stride=2, padding=1),
                               torch.nn.ReLU(),
                               torch.nn.BatchNorm2d(128),
                               # 8x8
                           
                               torch.nn.ConvTranspose2d(128, 128, 4, stride=2, padding=1),
                               torch.nn.ReLU(),
                               torch.nn.BatchNorm2d(128),
                               # 16x16
                           
                               torch.nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1),
                               torch.nn.ReLU(),
                               torch.nn.BatchNorm2d(64),
                               # 32x32
                           
                               torch.nn.ConvTranspose2d(64, 64, 4, stride=2, padding=1),
                               torch.nn.ReLU(),
                               torch.nn.BatchNorm2d(64),
                               # 64x64
                           
                               torch.nn.ConvTranspose2d(64, 2, 4, stride=2, padding=1), 
                               # 128x128
                           
                           torch.nn.LogSoftmax(dim=1)
                          )

In [ ]:
# определим модель с удобным функционалом

model = NNSegmentation(unet, lr=1e-3, criterion=torch.nn.NLLLoss())

In [ ]:
model.fit_loader(train_loader, epochs=50,
          valid_data=[X_test, y_test], log_every_epoch=10)

In [ ]:
model.show_history()

In [ ]:
# посмотрим на ошибку модели на тестовой части (чем меньше, тем лучше)
model.loss(X_test, y_test)

In [ ]:
model.show_predict_grid(X_test, y_test, size=3, threshold=0.2)